In [1]:
library(RPostgreSQL)
library(tableone)
library(tidyverse)

Loading required package: DBI
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
sql <-
"select hadm_id, icustay_id, echo,
    intime, outtime, echo_time, deathtime,
    norepinephrine_max, dobutamine_flag,
    vasofreeday28, ventfreeday28,
    sofa, sofa_drop_2, sofa_drop_3,
    day1, day2, day3, down2, down3,
    lactate_diff, creatinine_diff,
    lactate_diff_24, creatinine_diff_24
from (select hadm_id, icustay_id, echo, intime, outtime, echo_time, deathtime from merged_data) co
left join subgroup using (hadm_id, icustay_id, echo)
left join lactate_diff using (hadm_id)
left join creatinine_diff using (hadm_id)
left join (select hadm_id, lactate_diff as lactate_diff_24 from lactate_diff_24) lt_24 using (hadm_id)
left join (select hadm_id, creatinine_diff as creatinine_diff_24 from creatinine_diff_24) cr_24 using (hadm_id)
left join (select icustay_id, day1, day2, day3, down2, down3 from fluid) fl using (icustay_id)"

In [5]:
subgroup <- dbGetQuery(con, sql)
head(subgroup)

hadm_id,icustay_id,echo,intime,outtime,echo_time,deathtime,norepinephrine_max,dobutamine_flag,vasofreeday28,⋯,sofa_drop_3,day1,day2,day3,down2,down3,lactate_diff,creatinine_diff,lactate_diff_24,creatinine_diff_24
163557,200003,1,2199-08-02 19:50:04,2199-08-08 17:09:18,2199-08-03,2199-10-17 00:00:00,0.000000,0,26.68750,⋯,3,1710.4062,276.9948,-769.100,1433.41146,2479.506,1.6,0.0,1.3,-0.1
127229,200014,1,2105-02-16 23:16:48,2105-02-18 16:53:29,2105-02-18,2105-02-27 00:00:00,0.000000,0,0.00000,⋯,2,1516.0000,NA,NA,NA,NA,NA,-0.1,NA,-0.2
165426,200030,1,2150-11-13 14:08:02,2150-11-19 17:51:59,2150-11-16,2152-04-05 00:00:00,1.084800,1,26.29167,⋯,5,422.5490,327.0000,-2115.000,95.54901,2537.549,-1.1,0.0,-1.1,0.0
198650,200033,0,2198-08-07 17:56:17,2198-08-21 14:59:18,NA,2198-08-21 11:15:00,0.000000,0,0.00000,⋯,-5,720.7583,346.7278,2223.795,374.03047,-1503.037,-0.1,0.1,-0.4,0.0
159243,200036,0,2181-08-26 16:38:25,2181-08-29 16:28:48,NA,2184-11-13 00:00:00,0.000000,0,28.00000,⋯,7,3750.0000,905.0000,1601.000,2845.00000,2149.000,NA,0.5,NA,0.4
132255,200075,1,2159-09-23 00:13:20,2159-09-25 01:55:17,2159-09-25,NA,1.058141,0,27.66319,⋯,5,2616.9449,-1983.8871,NA,4600.83201,NA,NA,0.1,NA,0.0


In [6]:
nrow(subgroup)

[1] 6361

In [7]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [8]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,0.5525232,1.809879
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,1,1,0,0,0,0,1,0.2804613,1.389779
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,0.3012020,3.320031
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7982833,1.252688
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,1,1,0,0,0,0,0,0.2244447,1.289399
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3320492,3.011602


In [9]:
subgroup <- subgroup %>%
left_join(full_data %>% select(icustay_id, ps_weight, ps), by = "icustay_id") %>%
mutate(dobutamine_flag = as.factor(dobutamine_flag))
head(subgroup)

hadm_id,icustay_id,echo,intime,outtime,echo_time,deathtime,norepinephrine_max,dobutamine_flag,vasofreeday28,⋯,day2,day3,down2,down3,lactate_diff,creatinine_diff,lactate_diff_24,creatinine_diff_24,ps_weight,ps
163557,200003,1,2199-08-02 19:50:04,2199-08-08 17:09:18,2199-08-03,2199-10-17 00:00:00,0.000000,0,26.68750,⋯,276.9948,-769.100,1433.41146,2479.506,1.6,0.0,1.3,-0.1,1.981141,0.5047597
127229,200014,1,2105-02-16 23:16:48,2105-02-18 16:53:29,2105-02-18,2105-02-27 00:00:00,0.000000,0,0.00000,⋯,NA,NA,NA,NA,NA,-0.1,NA,-0.2,1.305681,0.7658838
165426,200030,1,2150-11-13 14:08:02,2150-11-19 17:51:59,2150-11-16,2152-04-05 00:00:00,1.084800,1,26.29167,⋯,327.0000,-2115.000,95.54901,2537.549,-1.1,0.0,-1.1,0.0,2.337547,0.4277989
198650,200033,0,2198-08-07 17:56:17,2198-08-21 14:59:18,NA,2198-08-21 11:15:00,0.000000,0,0.00000,⋯,346.7278,2223.795,374.03047,-1503.037,-0.1,0.1,-0.4,0.0,1.554794,0.3568281
159243,200036,0,2181-08-26 16:38:25,2181-08-29 16:28:48,NA,2184-11-13 00:00:00,0.000000,0,28.00000,⋯,905.0000,1601.000,2845.00000,2149.000,NA,0.5,NA,0.4,2.606293,0.6163132
132255,200075,1,2159-09-23 00:13:20,2159-09-25 01:55:17,2159-09-25,NA,1.058141,0,27.66319,⋯,-1983.8871,NA,4600.83201,NA,NA,0.1,NA,0.0,1.437882,0.6954673


In [10]:
features <- c("ventfreeday28", "vasofreeday28", "dobutamine_flag", "day1", "day2", "day3",
              "sofa_drop_2", "sofa_drop_3",
              "norepinephrine_max", "lactate_diff", "creatinine_diff",
              "lactate_diff_24", "creatinine_diff_24")
features

[1] "ventfreeday28"      "vasofreeday28"      "dobutamine_flag"   
 [4] "day1"               "day2"               "day3"              
 [7] "sofa_drop_2"        "sofa_drop_3"        "norepinephrine_max"
[10] "lactate_diff"       "creatinine_diff"    "lactate_diff_24"   
[13] "creatinine_diff_24"

In [11]:
missing <- subgroup %>%
select(features) %>%
summarize_all(funs(sum(is.na(.)) / n())) %>%
gather("cov", "na") %>%
mutate(na = (na * 100) %>% round(1) %>% paste0("%"))

missing

cov,na
ventfreeday28,0%
vasofreeday28,0%
dobutamine_flag,0%
day1,3.6%
day2,15.3%
day3,33%
sofa_drop_2,0%
sofa_drop_3,0%
norepinephrine_max,0%
lactate_diff,71.6%


In [12]:
tab <- CreateTableOne(vars = features, strata = "echo",
                      data = subgroup,
                      argsNormal = list(var.equal = FALSE))
capture.output(tab_df <- tab %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [13]:
tab_df

,0,1,p,SMD
n,3099,3262,,
ventfreeday28 (mean (sd)),19.09 (13.53),18.02 (25.13),0.033,0.053
vasofreeday28 (mean (sd)),20.25 (12.61),20.13 (14.91),0.741,0.008
dobutamine_flag = 1 (%),23 (0.7),137 (4.2),<0.001,0.224
day1 (mean (sd)),2303.55 (3670.67),2602.06 (4025.58),0.002,0.077
day2 (mean (sd)),964.10 (2645.46),1334.97 (3005.02),<0.001,0.131
day3 (mean (sd)),304.31 (2194.46),720.55 (2638.10),<0.001,0.172
sofa_drop_2 (mean (sd)),1.22 (4.34),0.89 (3.39),0.001,0.086
sofa_drop_3 (mean (sd)),2.29 (4.49),1.49 (4.07),<0.001,0.186
norepinephrine_max (mean (sd)),0.81 (2.39),1.78 (5.69),<0.001,0.221


In [14]:
ipw_svydesign <- survey::svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = subgroup)

In [15]:
tab_wtd <- svyCreateTableOne(vars = features,
                             strata = "echo",
                             data = ipw_svydesign)
capture.output(tab_wtd_df <- tab_wtd %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [16]:
tab_wtd_df

,0,1,p,SMD
n,5739.48,5842.36,,
ventfreeday28 (mean (sd)),14.78 (16.31),16.18 (36.90),0.903,0.003
vasofreeday28 (mean (sd)),18.13 (13.84),19.86 (20.23),0.047,0.054
dobutamine_flag = 1 (%),62.0 (1.1),196.3 (3.4),<0.001,0.155
day1 (mean (sd)),3578.20 (3784.46),3889.22 (4421.38),0.394,0.023
day2 (mean (sd)),2293.20 (2795.53),2484.97 (3305.21),0.001,0.100
day3 (mean (sd)),1077.95 (2689.29),1683.90 (3066.03),<0.001,0.170
sofa_drop_2 (mean (sd)),1.01 (2.83),0.55 (3.03),<0.001,0.128
sofa_drop_3 (mean (sd)),1.22 (3.32),0.78 (3.57),<0.001,0.247
norepinephrine_max (mean (sd)),1.77 (3.26),3.05 (8.49),<0.001,0.154


In [17]:
ps_matches_df <- data.table::fread(file.path(data_dir, "ps_matches_df.csv"), data.table = FALSE)
head(ps_matches_df)

icustay_id,group,match
262436,ctrl,1
244460,ctrl,2
298177,ctrl,3
290165,ctrl,4
278397,ctrl,5
212022,ctrl,6


In [18]:
ps_df <- ps_matches_df %>% left_join(subgroup, by = "icustay_id")
head(ps_df)

icustay_id,group,match,hadm_id,echo,intime,outtime,echo_time,deathtime,norepinephrine_max,⋯,day2,day3,down2,down3,lactate_diff,creatinine_diff,lactate_diff_24,creatinine_diff_24,ps_weight,ps
262436,ctrl,1,133275,0,2173-12-07 15:36:30,2173-12-09 21:27:39,NA,2173-12-09 18:30:00,4.46880,⋯,7625.3752,290.09175,-4232.334,3102.950,NA,NA,NA,-0.9,2.235326,0.5526379
244460,ctrl,2,177445,0,2143-12-27 20:59:00,2143-12-29 20:04:00,NA,NA,3.79920,⋯,1665.0000,NA,1457.250,NA,NA,0.3,NA,0.3,1.430385,0.3008876
298177,ctrl,3,133566,0,2166-07-17 16:15:16,2166-07-22 18:41:28,NA,2166-07-27 00:00:00,1.50273,⋯,996.6645,-52.54445,6913.559,7962.768,NA,0.1,NA,0.0,4.957195,0.7982730
290165,ctrl,4,197570,0,2129-12-24 05:33:11,2129-12-29 21:00:19,NA,NA,0.00000,⋯,3867.3365,2009.85431,2911.590,4769.072,NA,0.2,NA,0.2,1.498298,0.3325758
278397,ctrl,5,194360,0,2187-08-28 19:40:43,2187-08-29 19:18:14,NA,NA,0.00000,⋯,NA,NA,NA,NA,NA,0.4,NA,0.7,4.070197,0.7543116
212022,ctrl,6,159164,0,2141-03-23 19:59:04,2141-03-26 10:52:57,NA,2141-04-12 00:02:00,0.00000,⋯,2080.1075,-359.61866,-1339.657,1100.069,NA,0.2,-1.9,0.3,1.372052,0.2711645


In [19]:
tab_ps_pval_df <- features %>% map(function(feature) {
    df <- ps_df %>%
    select(!! as.name(feature), match, echo) %>%
    filter(!is.na(!! as.name(feature)))
    matched <- (df %>% filter(echo == 0) %>% rename(ctrl = (!! as.name(feature))) %>% select(ctrl, match)) %>%
    inner_join(df %>% filter(echo == 1) %>% rename(trtd = (!! as.name(feature))) %>% select(trtd, match),
              by = "match") %>%
    filter(!is.na(ctrl), !is.na(trtd))
    
    if (is.factor(ps_df %>% pull(feature))) {
        tab <- table(matched$trtd, matched$ctrl, dnn = c("Echo", "Non-Echo"))
        tst <- mcnemar.test(tab)
    } else {
        tst <- t.test(matched$trtd, matched$ctrl, paired = TRUE)
    }
    
    data.frame(cov = feature, pval = tst$p.val, stringsAsFactors = FALSE)
}) %>%
data.table::rbindlist() %>%
as.data.frame(stringsAsFactors = FALSE) %>%
mutate(pval = case_when(
    pval < 0.001 ~ "<0.001",
    TRUE ~ as.character(round(pval, 3))
)) %>%
rbind(data.frame(cov = "n", pval = "", stringsAsFactors = FALSE), .) %>%
setNames(c("cov", "p value (PS matching)"))

tab_ps_pval_df

cov,p value (PS matching)
n,
ventfreeday28,0.089
vasofreeday28,0.004
dobutamine_flag,0.077
day1,0.844
day2,0.033
day3,0.001
sofa_drop_2,<0.001
sofa_drop_3,<0.001
norepinephrine_max,<0.001


In [20]:
tab_ps <- CreateTableOne(vars = features, strata = "echo", data = ps_df)
capture.output(tab_ps_df <- tab_ps %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [21]:
tab_ps_df

,0,1,p,SMD
n,1626,1626,,
ventfreeday28 (mean (sd)),18.10 (14.70),19.60 (32.64),0.090,0.059
vasofreeday28 (mean (sd)),19.31 (12.73),20.81 (16.95),0.004,0.100
dobutamine_flag = 1 (%),20 (1.2),34 (2.1),0.074,0.067
day1 (mean (sd)),2552.61 (3907.32),2555.53 (3900.29),0.983,0.001
day2 (mean (sd)),1083.61 (2867.93),1336.38 (2925.28),0.022,0.087
day3 (mean (sd)),306.64 (2245.55),813.12 (2718.42),<0.001,0.203
sofa_drop_2 (mean (sd)),1.31 (4.55),0.68 (3.26),<0.001,0.159
sofa_drop_3 (mean (sd)),2.46 (4.76),1.15 (3.99),<0.001,0.299
norepinephrine_max (mean (sd)),1.04 (2.68),1.38 (3.13),0.001,0.117


In [22]:
tab_fmt <- tab_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, pval, smd) %>%
setNames(c("cov", "Non-Echo", "Echo", "p value", "SMD"))

tab_fmt

cov,Non-Echo,Echo,p value,SMD
n,3099,3262,,
ventfreeday28 (mean (sd)),19.09 (13.53),18.02 (25.13),0.033,0.053
vasofreeday28 (mean (sd)),20.25 (12.61),20.13 (14.91),0.741,0.008
dobutamine_flag = 1 (%),0.7%,4.2%,<0.001,0.224
day1 (mean (sd)),2303.55 (3670.67),2602.06 (4025.58),0.002,0.077
day2 (mean (sd)),964.10 (2645.46),1334.97 (3005.02),<0.001,0.131
day3 (mean (sd)),304.31 (2194.46),720.55 (2638.10),<0.001,0.172
sofa_drop_2 (mean (sd)),1.22 (4.34),0.89 (3.39),0.001,0.086
sofa_drop_3 (mean (sd)),2.29 (4.49),1.49 (4.07),<0.001,0.186
norepinephrine_max (mean (sd)),0.81 (2.39),1.78 (5.69),<0.001,0.221


In [23]:
tab_wtd_fmt <- tab_wtd_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, pval, smd) %>%
setNames(c("cov", "Non-Echo (weighted cohort)", "Echo (weighted cohort)",
           "p value (IPW)", "SMD (weighted cohort)"))

tab_wtd_fmt

cov,Non-Echo (weighted cohort),Echo (weighted cohort),p value (IPW),SMD (weighted cohort)
n,5739.48,5842.36,,
ventfreeday28 (mean (sd)),14.78 (16.31),16.18 (36.90),0.903,0.003
vasofreeday28 (mean (sd)),18.13 (13.84),19.86 (20.23),0.047,0.054
dobutamine_flag = 1 (%),1.1%,3.4%,<0.001,0.155
day1 (mean (sd)),3578.20 (3784.46),3889.22 (4421.38),0.394,0.023
day2 (mean (sd)),2293.20 (2795.53),2484.97 (3305.21),0.001,0.100
day3 (mean (sd)),1077.95 (2689.29),1683.90 (3066.03),<0.001,0.170
sofa_drop_2 (mean (sd)),1.01 (2.83),0.55 (3.03),<0.001,0.128
sofa_drop_3 (mean (sd)),1.22 (3.32),0.78 (3.57),<0.001,0.247
norepinephrine_max (mean (sd)),1.77 (3.26),3.05 (8.49),<0.001,0.154


In [24]:
tab_ps_fmt <- tab_ps_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, smd) %>%
setNames(c("cov", "Non-Echo (PS matching)", "Echo (PS matching)", "SMD (PS matching)"))

tab_ps_fmt

cov,Non-Echo (PS matching),Echo (PS matching),SMD (PS matching)
n,1626,1626,
ventfreeday28 (mean (sd)),18.10 (14.70),19.60 (32.64),0.059
vasofreeday28 (mean (sd)),19.31 (12.73),20.81 (16.95),0.100
dobutamine_flag = 1 (%),1.2%,2.1%,0.067
day1 (mean (sd)),2552.61 (3907.32),2555.53 (3900.29),0.001
day2 (mean (sd)),1083.61 (2867.93),1336.38 (2925.28),0.087
day3 (mean (sd)),306.64 (2245.55),813.12 (2718.42),0.203
sofa_drop_2 (mean (sd)),1.31 (4.55),0.68 (3.26),0.159
sofa_drop_3 (mean (sd)),2.46 (4.76),1.15 (3.99),0.299
norepinephrine_max (mean (sd)),1.04 (2.68),1.38 (3.13),0.117


In [25]:
tab_full <-
tab_fmt %>%
full_join(tab_wtd_fmt, by = "cov") %>%
full_join(tab_ps_fmt, by = "cov") %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = strsplit(cov, " ") %>% map_chr(first)) %>%
full_join(tab_ps_pval_df, by = "cov") %>%
full_join(missing %>% setNames(c("cov", "Missing data")), by = "cov") %>%
mutate(cov = case_when(
    cov == "ventfreeday28" ~ "Ventilation free days in 28 days",
    cov == "vasofreeday28" ~ "Vasopressor free days in 28 days",
    cov == "dobutamine_flag" ~ "Dobutamine Use",
    cov == "day1" ~ "IV Fluid Day 1 (mL)",
    cov == "day2" ~ "IV Fluid Day 2 (mL)",
    cov == "day3" ~ "IV Fluid Day 3 (mL)",
    cov == "sofa_drop_2" ~ "SOFA Reduction Day 2",
    cov == "sofa_drop_3" ~ "SOFA Reduction Day 3",
    cov == "norepinephrine_max" ~ "Norepinephrine (maximum dosage mg/min)",
    cov == "lactate_diff" ~ "Serum Lactate Reduction (48 hours)",
    cov == "creatinine_diff" ~ "Serum Creatinine Reduction (48 hours)",
    cov == "lactate_diff_24" ~ "Serum Lactate Reduction (24 hours)",
    cov == "creatinine_diff_24" ~ "Serum Creatinine Reduction (24 hours)",
    TRUE ~ cov
))

tab_full

cov,Non-Echo,Echo,p value,SMD,Non-Echo (weighted cohort),Echo (weighted cohort),p value (IPW),SMD (weighted cohort),Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),p value (PS matching),Missing data
n,3099,3262,,,5739.48,5842.36,,,1626,1626,,,NA
Ventilation free days in 28 days,19.09 (13.53),18.02 (25.13),0.033,0.053,14.78 (16.31),16.18 (36.90),0.903,0.003,18.10 (14.70),19.60 (32.64),0.059,0.089,0%
Vasopressor free days in 28 days,20.25 (12.61),20.13 (14.91),0.741,0.008,18.13 (13.84),19.86 (20.23),0.047,0.054,19.31 (12.73),20.81 (16.95),0.100,0.004,0%
Dobutamine Use,0.7%,4.2%,<0.001,0.224,1.1%,3.4%,<0.001,0.155,1.2%,2.1%,0.067,0.077,0%
IV Fluid Day 1 (mL),2303.55 (3670.67),2602.06 (4025.58),0.002,0.077,3578.20 (3784.46),3889.22 (4421.38),0.394,0.023,2552.61 (3907.32),2555.53 (3900.29),0.001,0.844,3.6%
IV Fluid Day 2 (mL),964.10 (2645.46),1334.97 (3005.02),<0.001,0.131,2293.20 (2795.53),2484.97 (3305.21),0.001,0.100,1083.61 (2867.93),1336.38 (2925.28),0.087,0.033,15.3%
IV Fluid Day 3 (mL),304.31 (2194.46),720.55 (2638.10),<0.001,0.172,1077.95 (2689.29),1683.90 (3066.03),<0.001,0.170,306.64 (2245.55),813.12 (2718.42),0.203,0.001,33%
SOFA Reduction Day 2,1.22 (4.34),0.89 (3.39),0.001,0.086,1.01 (2.83),0.55 (3.03),<0.001,0.128,1.31 (4.55),0.68 (3.26),0.159,<0.001,0%
SOFA Reduction Day 3,2.29 (4.49),1.49 (4.07),<0.001,0.186,1.22 (3.32),0.78 (3.57),<0.001,0.247,2.46 (4.76),1.15 (3.99),0.299,<0.001,0%
Norepinephrine (maximum dosage mg/min),0.81 (2.39),1.78 (5.69),<0.001,0.221,1.77 (3.26),3.05 (8.49),<0.001,0.154,1.04 (2.68),1.38 (3.13),0.117,<0.001,0%


In [26]:
tab_report <- tab_full %>% select(1, 10, 11, 12, 13)
tab_report

cov,Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),p value (PS matching)
n,1626,1626,,
Ventilation free days in 28 days,18.10 (14.70),19.60 (32.64),0.059,0.089
Vasopressor free days in 28 days,19.31 (12.73),20.81 (16.95),0.100,0.004
Dobutamine Use,1.2%,2.1%,0.067,0.077
IV Fluid Day 1 (mL),2552.61 (3907.32),2555.53 (3900.29),0.001,0.844
IV Fluid Day 2 (mL),1083.61 (2867.93),1336.38 (2925.28),0.087,0.033
IV Fluid Day 3 (mL),306.64 (2245.55),813.12 (2718.42),0.203,0.001
SOFA Reduction Day 2,1.31 (4.55),0.68 (3.26),0.159,<0.001
SOFA Reduction Day 3,2.46 (4.76),1.15 (3.99),0.299,<0.001
Norepinephrine (maximum dosage mg/min),1.04 (2.68),1.38 (3.13),0.117,<0.001


In [27]:
data.table::fwrite(tab_full, file.path(data_dir, "subgroup.csv"))
data.table::fwrite(tab_report, file.path(data_dir, "subgroup_report.csv"))